In [1]:
import pandas as pd
import glob
import altair as alt
from sublimpy import utils
import datetime as dt
import numpy as np

In [2]:
files = glob.glob("turb_datasets/tidy_df**.parquet")
df = pd.DataFrame()
for file in files:
    local = pd.read_parquet(file).query("measurement == 'w_h2o_'")
    df = pd.concat([df, local.assign(filename = file)])
df = df[df.variable.isin([
    'w_h2o__2m_c_gapfill', 'w_h2o__3m_c_gapfill', 'w_h2o__5m_c_gapfill', 'w_h2o__10m_c_gapfill', 'w_h2o__15m_c_gapfill', 'w_h2o__20m_c_gapfill', 
    'w_h2o__3m_uw_gapfill', 'w_h2o__10m_uw_gapfill', 
    'w_h2o__3m_ue_gapfill', 'w_h2o__10m_ue_gapfill', 
    'w_h2o__3m_d_gapfill', 'w_h2o__10m_d_gapfill', 
])]
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')
df = df[(df['time'] > '20221130') & (df['time'] < '20230508')]

In [3]:
df['filename'] = df['filename'].str.replace('turb_datasets/tidy_df_20221101_20230619_planar_fit_multiplane_STRAIGHTUP_', '')
df['processing'] = df['filename'].str.replace('.parquet', '')

In [4]:
df.head()

,time,variable,value,height,tower,measurement,filename,processing
4583767,2022-11-30 00:30:00,w_h2o__2m_c_gapfill,-0.000715,2.0,c,w_h2o_,q7_flags3600_snowfallfilteredno.parquet,q7_flags3600_snowfallfilteredno
4583768,2022-11-30 01:00:00,w_h2o__2m_c_gapfill,-0.000539,2.0,c,w_h2o_,q7_flags3600_snowfallfilteredno.parquet,q7_flags3600_snowfallfilteredno
4583769,2022-11-30 01:30:00,w_h2o__2m_c_gapfill,-0.000337,2.0,c,w_h2o_,q7_flags3600_snowfallfilteredno.parquet,q7_flags3600_snowfallfilteredno
4583770,2022-11-30 02:00:00,w_h2o__2m_c_gapfill,-0.000885,2.0,c,w_h2o_,q7_flags3600_snowfallfilteredno.parquet,q7_flags3600_snowfallfilteredno
4583771,2022-11-30 02:30:00,w_h2o__2m_c_gapfill,-0.000072,2.0,c,w_h2o_,q7_flags3600_snowfallfilteredno.parquet,q7_flags3600_snowfallfilteredno


# Calculate and plot seasonal totals

In [5]:
totals_df = pd.DataFrame(df.groupby(['variable', 'processing'])['value'].sum()*30*60/1000).reset_index()

In [6]:
totals_df['despiking'] = totals_df['processing'].apply(lambda s: s.split('_')[0])
totals_df['instrument_flagging'] = totals_df['processing'].apply(lambda s: s.split('_')[1])
totals_df['snowfall_filtering'] = totals_df['processing'].apply(lambda s: s.split('_')[2])
totals_df['height'] = totals_df['variable'].str.split('_').str[3].str[:-1].astype(int)
totals_df['tower'] = totals_df['variable'].str.split('_').str[-1]
totals_df

,variable,processing,value,despiking,instrument_flagging,snowfall_filtering,height,tower
0,w_h2o__10m_c_gapfill,nodespiking_flags36000_snowfallfilteredno,41.066913,nodespiking,flags36000,snowfallfilteredno,10,gapfill
1,w_h2o__10m_c_gapfill,nodespiking_flags3600_snowfallfilteredno,38.771281,nodespiking,flags3600,snowfallfilteredno,10,gapfill
2,w_h2o__10m_c_gapfill,nodespiking_flags9000_snowfallfilteredno,39.934237,nodespiking,flags9000,snowfallfilteredno,10,gapfill
3,w_h2o__10m_c_gapfill,q7_flags36000_snowfallfilteredno,39.038130,q7,flags36000,snowfallfilteredno,10,gapfill
4,w_h2o__10m_c_gapfill,q7_flags3600_snowfallfilteredno,37.103656,q7,flags3600,snowfallfilteredno,10,gapfill
...,...,...,...,...,...,...,...,...
67,w_h2o__5m_c_gapfill,nodespiking_flags3600_snowfallfilteredno,37.734793,nodespiking,flags3600,snowfallfilteredno,5,gapfill
68,w_h2o__5m_c_gapfill,nodespiking_flags9000_snowfallfilteredno,38.140115,nodespiking,flags9000,snowfallfilteredno,5,gapfill
69,w_h2o__5m_c_gapfill,q7_flags36000_snowfallfilteredno,36.930368,q7,flags36000,snowfallfilteredno,5,gapfill
70,w_h2o__5m_c_gapfill,q7_flags3600_snowfallfilteredno,36.010166,q7,flags3600,snowfallfilteredno,5,gapfill


In [7]:
alt.Chart(totals_df).mark_point(size=100).encode(
    alt.X('value:Q').scale(zero=False, domain=[20,40]),
    alt.Y('height:Q'),
    alt.Shape('tower:N'),
    alt.Column('instrument_flagging').sort(['flags36000', 'flags9000', 'flags3600']),
    alt.Row('despiking'),
).properties(width=150, height = 150)

alt.Chart(...)

In [10]:
totals_df.snowfall_filtering.unique()

array(['snowfallfilteredno'], dtype=object)

In [26]:
values_9000 = totals_df[totals_df.despiking == 'q7'].query("instrument_flagging == 'flags9000'").set_index('variable')['value']
values_3600 = totals_df[totals_df.despiking == 'q7'].query("instrument_flagging == 'flags3600'").set_index('variable')['value']
changes_due_to_flagging = round(
    ((values_9000 - values_3600)/values_9000)*100,
    1
)
print(changes_due_to_flagging.mean(), changes_due_to_flagging.median())
changes_due_to_flagging

1.0333333333333332 1.2


variable
w_h2o__10m_c_gapfill     3.1
w_h2o__10m_d_gapfill     2.3
w_h2o__10m_ue_gapfill    2.8
w_h2o__10m_uw_gapfill    0.8
w_h2o__15m_c_gapfill    -5.0
w_h2o__20m_c_gapfill     0.6
w_h2o__2m_c_gapfill      2.3
w_h2o__3m_c_gapfill      1.4
w_h2o__3m_d_gapfill      0.2
w_h2o__3m_ue_gapfill     1.9
w_h2o__3m_uw_gapfill     1.0
w_h2o__5m_c_gapfill      1.0
Name: value, dtype: float64

In [151]:
src = totals_df.assign(variable = totals_df.variable.apply(lambda s: s[:-8]))
y = alt.Y('variable:N', sort = [
        'w_h2o__20m_c', 'w_h2o__15m_c', 
        'w_h2o__10m_c', 'w_h2o__10m_d', 'w_h2o__10m_ue', 'w_h2o__10m_uw',
        'w_h2o__5m_c',
        'w_h2o__3m_c', 'w_h2o__3m_d', 'w_h2o__3m_ue', 'w_h2o__3m_uw', 
        'w_h2o__2m_c',
    ])
alt.Chart(
    src
).mark_rule().encode(
    alt.X('min(value):Q').scale(zero=False),
    alt.X2('max(value):Q'),
    y,
) + alt.Chart(
    src
).transform_filter(
    "datum.processing == 'q7_flags9000_snowfallfilteredno'"
).mark_circle(color='blue', opacity=1).encode(
    alt.X('value:Q').title([
            'Cumulative sublimation (mm SWE)',

        ]),
    y,
).properties(width = 200, height = 200)

alt.LayerChart(...)

In [110]:
alt.Chart(
    src
).mark_rule().encode(
    alt.X('min(value):Q').scale(zero=False).title('Cumulative sublimation (mm SWE)'),
    alt.X2('max(value):Q'),
    alt.Y('despiking:N').title(''),
    alt.Row('variable').title('').header(labelAngle=0, labelPadding=-60),
).properties(width=200, height=50) | alt.Chart(
    src
).mark_rule().encode(
    alt.X('min(value):Q').scale(zero=False).title('Cumulative sublimation (mm SWE)'),
    alt.X2('max(value):Q'),
    alt.Y('instrument_flagging:N').title(''),
    alt.Row('variable').title('').header(labelAngle=0, labelPadding=-60),
).properties(width=200, height=50)

alt.HConcatChart(...)

In [59]:
round(100 * (
    totals_df.groupby('variable')['value'].max() 
    - totals_df.groupby('variable')['value'].min()
) / totals_df.groupby('variable')['value'].min(), 1)

variable
w_h2o__10m_c_gapfill     10.7
w_h2o__10m_d_gapfill      7.2
w_h2o__10m_ue_gapfill     7.0
w_h2o__10m_uw_gapfill     6.2
w_h2o__15m_c_gapfill      9.2
w_h2o__20m_c_gapfill      2.7
w_h2o__2m_c_gapfill       4.0
w_h2o__3m_c_gapfill       7.4
w_h2o__3m_d_gapfill       1.6
w_h2o__3m_ue_gapfill      7.5
w_h2o__3m_uw_gapfill      5.4
w_h2o__5m_c_gapfill       7.6
Name: value, dtype: float64

In [113]:
print(totals_df.despiking.unique())
print(totals_df.instrument_flagging.unique())

['nodespiking' 'q7']
['flags36000' 'flags3600' 'flags9000']


In [126]:
(round(100 * (
    totals_df.query("despiking == 'nodespiking'").groupby('variable')['value'].max() 
    - totals_df.query("despiking == 'nodespiking'").groupby('variable')['value'].min()
) / totals_df.query("despiking == 'nodespiking'").groupby('variable')['value'].min(), 1)).mean()

3.483333333333334

In [120]:
(round(100 * (
    totals_df.query("despiking == 'q7'").groupby('variable')['value'].max() 
    - totals_df.query("despiking == 'q7'").groupby('variable')['value'].min()
) / totals_df.query("despiking == 'q7'").groupby('variable')['value'].min(), 1)).mean()

4.016666666666667

In [121]:
(round(100 * (
    totals_df.query("instrument_flagging == 'flags36000'").groupby('variable')['value'].max() 
    - totals_df.query("instrument_flagging == 'flags36000'").groupby('variable')['value'].min()
) / totals_df.query("instrument_flagging == 'flags36000'").groupby('variable')['value'].min(), 1)).mean()

2.5083333333333333

In [122]:
(round(100 * (
    totals_df.query("instrument_flagging == 'flags9000'").groupby('variable')['value'].max() 
    - totals_df.query("instrument_flagging == 'flags9000'").groupby('variable')['value'].min()
) / totals_df.query("instrument_flagging == 'flags9000'").groupby('variable')['value'].min(), 1)).mean()

2.6416666666666666

In [123]:
(round(100 * (
    totals_df.query("instrument_flagging == 'flags3600'").groupby('variable')['value'].max() 
    - totals_df.query("instrument_flagging == 'flags3600'").groupby('variable')['value'].min()
) / totals_df.query("instrument_flagging == 'flags3600'").groupby('variable')['value'].min(), 1)).mean()

2.5749999999999997

# Calculate and examine seasonal cumulative

In [ ]:
alt.data_transformers.enable('json')

In [ ]:
VARIABLE = 'w_h2o__20m_c'
processing_options = df.processing.unique()
local = pd.DataFrame()
for process in processing_options:
    local = pd.concat([
        local,
        df.query(f"variable == '{VARIABLE}'").query(f"processing == '{process}'").set_index('time').sort_index()[['value']].cumsum().assign(processing = process)
    ])
local['value'] = local['value']*1.8
local['despiking'] = local['processing'].apply(lambda s: s.split('_')[0])
local['instrument_flagging'] = local['processing'].apply(lambda s: s.split('_')[1])
local['snowfall_filtering'] = local['processing'].apply(lambda s: s.split('_')[2])

In [ ]:
alt.Chart(local.reset_index().dropna()).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('processing:N'),
    alt.Row('snowfall_filtering:N'),
    tooltip = 'processing:N'
).properties(width=1000).resolve_scale(color='independent')

In [ ]:
pd.options.display.max_rows = 500
display(df.groupby(['variable','processing']).value.max())

In [ ]:
def restrict_time(src):
    return src[
        (
            src.time > '20230111'
        )&(
            src.time < '20230111 0600'
        )
    ]
alt.Chart(restrict_time(
    df.query("variable == 'w_h2o__20m_c'")
)).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('processing'),
    facet='processing'
)

In [ ]:
def restrict_time(src):
    return src[
        (
            src.time > '20221221'
        )&(
            src.time < '20221223'
        )
    ]
alt.Chart(restrict_time(
    df.query("variable == 'w_h2o__20m_c'")
)).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('processing'),
    facet='processing'
)

In [ ]:
restrict_time(
    df.query("variable == 'w_h2o__20m_c'")
).value.value_counts()

# Apply mean diurnal gap filling 

In [ ]:
df_gapfilled = pd.DataFrame()
for lhflux_variable in df.variable.unique():
    for process in df.processing.unique():
        subset = df.query(f"variable == '{lhflux_variable}'")
        subset = subset.query(f"processing == '{process}'")
        subset = subset.set_index('time')
        for i,row in subset.iterrows():
            if np.isnan(row['value']) or row['value'] == 0:
                start_window = i - dt.timedelta(days=3, hours=12)
                end_window = i + dt.timedelta(days=3, hours=12)
                src = subset.loc[start_window: end_window].reset_index()
                means = pd.DataFrame(
                    src.groupby([src.time.dt.hour, src.time.dt.minute])['value'].mean()
                )
                subset.loc[i, 'value'] = means.loc[i.hour, i.minute].value
        df_gapfilled = pd.concat([df_gapfilled, subset])

In [ ]:
totals_gapfilled_df = pd.DataFrame(df_gapfilled.groupby(['variable', 'processing'])['value'].sum()*30*60/1000).reset_index()

In [ ]:
totals_gapfilled_df['despiking'] = totals_gapfilled_df['processing'].apply(lambda s: s.split('_')[0])
totals_gapfilled_df['instrument_flagging'] = totals_gapfilled_df['processing'].apply(lambda s: s.split('_')[1])
totals_gapfilled_df['snowfall_filtering'] = totals_gapfilled_df['processing'].apply(lambda s: s.split('_')[2])
totals_gapfilled_df['height'] = totals_gapfilled_df['variable'].str.split('_').str[3].str[:-1].astype(int)
totals_gapfilled_df['tower'] = totals_gapfilled_df['variable'].str.split('_').str[-1]
totals_gapfilled_df

In [ ]:
alt.Chart(totals_gapfilled_df).mark_point(size=100).encode(
    alt.X('value:Q').scale(zero=False, domain=[20,40]),
    alt.Y('height:Q'),
    alt.Shape('tower:N'),
    alt.Column('instrument_flagging').sort(['flags36000', 'flags9000', 'flags3600']),
    alt.Row('despiking'),
    alt.Color('snowfall_filtering')
).properties(width=150, height = 150).display(renderer='svg')

In [ ]:
def restrict_time(src):
    return src[
        (
            src.time > '20221221'
        )&(
            src.time < '20221223'
        )
    ]
alt.Chart(restrict_time(
    df.query("variable == 'w_h2o__20m_c'")
)).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('processing'),
    facet='processing'
)